### clone代码

In [ ]:
!git clone https://github.com/modelscope/modelscope-agent.git

### 安装特定依赖

In [ ]:
!cd modelscope-agent && pip install -r requirements.txt

### 本地配置

In [ ]:
import logging
import sys
import os

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
os.chdir('modelscope-agent/demo')
sys.path.append('../')

### 导入llama_index相关依赖

In [ ]:
from IPython.display import Markdown, display
import torch
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts import PromptTemplate
from modelscope import snapshot_download
from llama_index.core.base.embeddings.base import BaseEmbedding, Embedding
from abc import ABC
from typing import Any, List, Optional, Dict, cast
from llama_index.core import (
    VectorStoreIndex,
    ServiceContext,
    set_global_service_context,
    SimpleDirectoryReader,
)

### 本地搭建知识库llm

In [ ]:
# Model names 
qwen2_4B_CHAT = "qwen/Qwen1.5-4B-Chat"

selected_model = snapshot_download(qwen2_4B_CHAT)

SYSTEM_PROMPT = """You are a helpful AI assistant.
"""

query_wrapper_prompt = PromptTemplate(
    "[INST]<<SYS>>\n" + SYSTEM_PROMPT + "<</SYS>>\n\n{query_str}[/INST] "
)

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=2048,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name=selected_model,
    model_name=selected_model,
    device_map="auto",
    # change these settings below depending on your GPU
    model_kwargs={"torch_dtype": torch.float16},
)

### 下载知识库文件

In [ ]:
!mkdir -p 'demo/data/xianjiaoda/'
!wget 'https://modelscope.oss-cn-beijing.aliyuncs.com/resource/rag/xianjiaoda.md' -O 'data/xianjiaoda/xianjiaoda.md'

### 导入知识库文件

In [ ]:
# load documents
documents = SimpleDirectoryReader("demo/data/xianjiaoda/").load_data()
documents

### 构建gte embedding

In [ ]:
embedding_model = "damo/nlp_gte_sentence-embedding_chinese-base"
class ModelScopeEmbeddings4LlamaIndex(BaseEmbedding, ABC):
    embed: Any = None
    model_id: str = "damo/nlp_gte_sentence-embedding_chinese-base"

    def __init__(
            self,
            model_id: str,
            **kwargs: Any,
    ) -> None:
        super().__init__(**kwargs)
        try:
            from modelscope.models import Model
            from modelscope.pipelines import pipeline
            from modelscope.utils.constant import Tasks
            # 使用modelscope的embedding模型（包含下载）
            self.embed = pipeline(Tasks.sentence_embedding, model=self.model_id)

        except ImportError as e:
            raise ValueError(
                "Could not import some python packages." "Please install it with `pip install modelscope`."
            ) from e

    def _get_query_embedding(self, query: str) -> List[float]:
        text = query.replace("\n", " ")
        inputs = {"source_sentence": [text]}
        return self.embed(input=inputs)['text_embedding'][0].tolist()

    def _get_text_embedding(self, text: str) -> List[float]:
        text = text.replace("\n", " ")
        inputs = {"source_sentence": [text]}
        return self.embed(input=inputs)['text_embedding'][0].tolist()

    def _get_text_embeddings(self, texts: List[str]) -> List[List[float]]:
        texts = list(map(lambda x: x.replace("\n", " "), texts))
        inputs = {"source_sentence": texts}
        return self.embed(input=inputs)['text_embedding'].tolist()

    async def _aget_query_embedding(self, query: str) -> List[float]:
        return self._get_query_embedding(query)

### 构建LlamaIndex服务

In [ ]:
embeddings = ModelScopeEmbeddings4LlamaIndex(model_id=embedding_model)
service_context = ServiceContext.from_defaults(embed_model=embeddings, llm=llm)
set_global_service_context(service_context)

index = VectorStoreIndex.from_documents(documents)
# set Logging to DEBUG for more detailed outputs
query_engine = index.as_query_engine()

### 测试LlamaIndex服务

In [ ]:
response = query_engine.query(str_or_query_bundle="西安交大是由哪几个学校合并的?")
print(response)

### API_KEY管理

In [ ]:
import os

print('请输入DASHSCOPE_API_KEY')
os.environ['DASHSCOPE_API_KEY'] = input()

### 工具注册

In [ ]:
from modelscope_agent.tools.base import BaseTool
from modelscope_agent.tools import register_tool

@register_tool('LlamaIndex-tool')
class CoupletGenerationTool(BaseTool):
    description = '查询本地知识库进行问题的回答'
    name = 'LlamaIndex-tool'
    parameters: list = [{
        'name': 'str_or_query_bundle',
        'description': '基于本地知识库来回答的问题',
        'required': True,
        'type': 'string'
    }]
    
    def call(self, params: str, **kwargs):
        params = self._verify_args(params)
        response = query_engine.query(**params)
        res = str(response)
        return res

### 构建agent

In [ ]:
from modelscope_agent.agents import RolePlay

role_template = '知识库查询小助手，可以优先通过查询本地知识库来回答用户的问题'
llm_config = {
    'model': 'qwen-max', 
    'model_server': 'dashscope',
    }
function_list = ['LlamaIndex-tool']

bot = RolePlay(function_list=function_list,llm=llm_config, instruction=role_template)

### 执行agent

In [ ]:
response = bot.run("西安交通大学现在有多少名学生？", remote=False, print_info=True)
text = ''
for chunk in response:
    text += chunk
print(text)